In [40]:
import numpy as np # linear algebra
import pandas as pd
import requests
from datetime import datetime
import time

In [3]:
# Đọc file CSV
file_path = '../data/raw/links.csv'
df = pd.read_csv(file_path)
df

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
87580,292731,26812510,1032473.0
87581,292737,14907358,986674.0
87582,292753,12388280,948139.0
87583,292755,64027,182776.0


In [4]:
df['imdbId'] = df['imdbId'].apply(lambda x: f'tt{str(x).zfill(7)}')
df

,movieId,imdbId,tmdbId
0,1,tt0114709,862.0
1,2,tt0113497,8844.0
2,3,tt0113228,15602.0
3,4,tt0114885,31357.0
4,5,tt0113041,11862.0
...,...,...,...
87580,292731,tt26812510,1032473.0
87581,292737,tt14907358,986674.0
87582,292753,tt12388280,948139.0
87583,292755,tt0064027,182776.0


In [21]:
# api_key_list = ['2d918779', '817e9847', '6d85304', 'c05a60d1', '7c616a06', 'f8711aaf', '8e22b5b']
# api_key_list = ['7c616a06', 'f8711aaf', '8e22b5b']
api_key_list = ['431d9c38', 'db7dff22', '3ff68b38']
# url_template = 'https://www.omdbapi.com/?i={}&apikey=' + api_key

In [38]:
   def get_movie_data(imdb_id, api_key):
    url = f"http://www.omdbapi.com/?i={imdb_id}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # Trả về dữ liệu JSON từ OMDB API
    else:
        return None

In [34]:
# Chia DataFrame thành các khối với mỗi khối 1000 dòng
def crawl_movies_by_api_keys(df, api_key_list):
    start = 0
    chunk_size = 2
    for i, api_key in enumerate(api_key_list):
        # Xác định phần của DataFrame dựa trên vị trí
        chunk = df.iloc[start:start + chunk_size]
        
        # Nếu chunk rỗng, nghĩa là hết dữ liệu để crawl
        if chunk.empty:
            break
        
        # Crawl thông tin phim cho từng imdbId trong chunk
        for imdb_id in chunk['imdbId']:
            movie_info = get_movie_data(imdb_id, api_key)
            if movie_info and movie_info.get('Response') == 'True':
                movie_data.append(movie_info)
            time.sleep(1)  # Tránh bị chặn API
        
        # Cập nhật vị trí bắt đầu cho chunk tiếp theo
        start += chunk_size

In [39]:
# Danh sách để lưu thông tin phim
movie_data = []

# Gọi hàm để crawl dữ liệu phim
crawl_movies_by_api_keys(df, api_key_list)

# Chuyển đổi dữ liệu sang DataFrame
df_movies = pd.DataFrame(movie_data)

# Lưu dữ liệu phim vào file CSV mới
output_file = 'movie_data.csv'
df_movies.to_csv(output_file, index=False)

print(f"Đã lưu thông tin phim vào file: {output_file}")

Đã lưu thông tin phim vào file: movie_data.csv


In [ ]:
import pandas as pd

In [9]:
df_raw = pd.read_csv("../data/raw/movie_data.csv")

/var/folders/70/8llpvtf936971vgt3bntcs3c0000gn/T/ipykernel_98150/1603638480.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("../data/raw/movie_data.csv")


In [10]:
df_raw

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Type,DVD,BoxOffice,Production,Website,Response,Season,Episode,seriesID,totalSeasons
0,Good Neighbor Sam,1964,Approved,22-Jul-64,130 min,Comedy,David Swift,"James Fritzell, Everett Greenbaum, David Swift","Jack Lemmon, Romy Schneider, Dorothy Provine",To help his divorced neighbor claim a substant...,...,movie,NaN,"$9,072,726",NaN,NaN,True,NaN,NaN,NaN,NaN
1,Eddie Izzard: Dress to Kill,1999,Unrated,13-Jun-99,115 min,"Documentary, Comedy",Lawrence Jordan,Eddie Izzard,Eddie Izzard,Executive transvestite Eddie Izzard takes his ...,...,movie,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
2,Quo Vadis,1951,Approved,25-Dec-51,171 min,"Biography, Drama, Romance","Mervyn LeRoy, Anthony Mann","John Lee Mahin, S.N. Behrman, Sonya Levien","Robert Taylor, Deborah Kerr, Leo Genn",After fierce Roman commander Marcus Vinicius b...,...,movie,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
3,The Captain's Paradise,1953,Approved,28-Sep-53,94 min,"Comedy, Romance",Anthony Kimmins,"Alec Coppel, Nicholas Phipps","Alec Guinness, Peter Bull, Charles Goldner",The captain of a ferry boat between the restri...,...,movie,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
4,Batman Begins,2005,PG-13,15-Jun-05,140 min,"Action, Crime, Drama",Christopher Nolan,"Bob Kane, David S. Goyer, Christopher Nolan","Christian Bale, Michael Caine, Ken Watanabe","After witnessing his parents' death, Bruce lea...",...,movie,NaN,"$206,863,479",NaN,NaN,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,Labrador,2011,NaN,30-Apr-11,73 min,Drama,Frederikke Aspöck,Daniel Dencik,"Carsten Bjørnlund, Stephanie Leon, Jakob Eklund","On a desolate, windswept island, Stella and Os...",...,movie,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
88510,Dead of the Nite,2013,Not Rated,07-Jan-14,86 min,Horror,S.J. Evans,S.J. Evans,"Tony Todd, Joseph Millson, Cicely Tennant",When a group of ghost hunters investigate the ...,...,movie,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
88511,Moscow Laughs,1934,NaN,24-Mar-35,96 min,"Comedy, Musical",Grigoriy Aleksandrov,"Grigoriy Aleksandrov, Nikolay Erdman, Vladimir...","Leonid Utyosov, Lyubov Orlova, Mariya Strelkova",A musically talented shepherd gets his big cha...,...,movie,NaN,NaN,Amkino Corporation,NaN,True,NaN,NaN,NaN,NaN
88512,Dead Tone,2007,R,24-Aug-07,100 min,"Horror, Thriller","Brian Hooks, Deon Taylor","Vashon Nutt, Brian Hooks, Deon Taylor","Austin Basis, Chris Blasman, Jonathan Chase",Finals at the prestigious University of Dreysk...,...,movie,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
